### gemma-2b-it Instruction Tuning
preprocess에서 전처리한 데이터를 이용해 gemma 모델을 학습시킵니다.
코드들은 [Stanford alpaca](https://github.com/tatsu-lab/stanford_alpaca)를 기반으로 작성되었습니다.
학습 환경은 아래와 같습니다.

|구분|내용|
|-|-|
|학습환경|Google Colab|
|GPU|L4(22.5GB)|
|VRAM|약 17GB 소요|
|dtype|bfloat16|
|Attention|flash attention2|
|Tuning|Lora(r=4, alpha=32)|
|Learning Rate|1e-4|
|Optimizer|adamw_torch_fused|

---

1. Pretrained 모델 설정  
GPU 환경에서는 [flash attention](https://github.com/Dao-AILab/flash-attention)을 사용하는 것이 좋습니다.
device는 cuda로 설정, dtype은 bfloat16으로 설정하였습니다.

2. 토크나이저를 정의합니다.  
추후 작성할 프롬프트들을 토크나이징하여 input_ids와 labels를 생성합니다.  

3. 마스킹
프롬프트들을 포맷팅한 다음 토크나이징 작업을 수행합니다.  
학습에 사용할 데이터들을 마스킹 작업하여 훈련에 사용할 수 있게 합니다.

4. `Dataset`과 `data_collator`를 정의합니다.  
2번의 토크나이징한 데이터로 `Dataset`을 생성하여 `train_data_set`으로 설정합니다.
`data_collator`를 통해 Sequence의 Padding을 수행합니다.

5. Lora 설정
Peft 학습을 위해 LoraConfig를 설정합니다.

6. 학습
1번의 Pretrained 모델, 4번의 Dataset 및 data_collator, 5번의 LoraConfig를 토대로 학습을 수행합니다.

In [1]:
!pip install --quiet\
selenium\
openai\
colorama\
datasets\
accelerate==0.27.2\
flash-attn\
peft\
trl\
transformers\
python-dotenv

In [3]:
import os
import sys
import json
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import torch
from datasets import Dataset
from google.colab import userdata, drive
drive.mount('/content/drive')

sys.path.append("/content/drive/MyDrive/Colab Notebooks/instruction-tuning-with-rag-example-main")
import utils
import prompts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Pretrained 모델 설정
[google/gemma-1.1-2b-it](https://huggingface.co/google/gemma-2b-it)  
구글 Colab Secret 기능으로 허깅페이스 토큰을 사용할 수 있습니다.([Colab Secret 가이드](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75))  
gemma 모델은 토큰이 없으면 받을 수 없습니다.  

학습시 메모리 사용량을 크게 줄일 수 있는 방법은 3가지 정도였습니다.
+ Quantization
+ Flash Attention
+ PEFT

#### Quantization
`torch.bfloat16` 또는 `torch.float16` 타입으로 퍼포먼스를 비교적 떨어뜨리지 않으면서 적은 메모리를 사용할 수 있습니다.  
온디바이스 AI가 아니라면 `int` 타입은 퍼포먼스 차이가 있으므로 유의해야합니다.  
더 자세한 내용은 [huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/performance#floating-data-types)를 참고하세요.  

#### Flash Attention
GPU를 사용하는 환경이면 flash attention을 사용하길 추천드립니다.  
inference시 15GB정도의 VRAM이 소요되었지만, flash attention을 사용시 5GB 정도로 VRAM 사용량이 감소하였습니다.  
flash attention은 속도와 메모리 사용량을 감소시킬 수 있지만 퍼포먼스의 큰 차이가 없습니다.
자세한 내용은 [논문](https://arxiv.org/pdf/2307.08691https://arxiv.org/pdf/2307.08691)을 참조해주세요.

#### PEFT
가장 많이 사용되는 LoRA의 경우 논문에 따르면 GPU 사용량이 1/3 가량으로 절감되었다고 합니다.  
또한 Full Fine Tuning과 비교하여 퍼포먼스의 차이도 크지 않다고 합니다(어떤 경우는 퍼포먼스가 더 좋았다고 합니다).
자세한 내용은 [논문](https://arxiv.org/pdf/2106.09685)를 참조해주세요.

In [18]:
model_id = "google/gemma-1.1-2b-it"
dtype = torch.bfloat16
token = userdata.get('HF_TOKEN_READ') # Colab Secret에 설정 가능
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
    token=token,
    attn_implementation="flash_attention_2"
)
# 토크나이저의 max_length가 매우 큰수로 지정되어있으므로, 모델의 sequence length인 8192로 세팅
tokenizer.model_max_length = model.config.max_position_embeddings

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 2. tokenizer 설정
프롬프트들을 받아 Embedding index를 반환하는 토크나이저 기능을 정의합니다.

In [7]:
def tokenize(strings, tokenizer):
    tokenized_list = [
        tokenizer(
            text,
            max_length=tokenizer.model_max_length,
            return_tensors="pt",
            padding="longest",
            truncation=True
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

### 3. 마스킹
데이터셋은 아래의 예시와 같이 전달됩니다.  
+ `input_ids`
```python
tensor([  28693, 238264, 236648, 234541,  90621, 235248, 235284, 237029,  47555,
        235265, 171066,  90621, 150483, 236800, 238597,  74715, 239618, 236137,
          ...
         95917,  96564,  75500, 237433, 235248, 235284, 237936, 237699,  31087,
         96564,  83160, 237036, 149735, 179694, 235265,    108,      1])
```

+ `labels`
```python
tensor([  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          ...
         95917,  96564,  75500, 237433, 235248, 235284, 237936, 237699,  31087,
         96564,  83160, 237036, 149735, 179694, 235265,    108,      1])
```

Stanford Alpaca의 예시를 영문으로 변경하면 아래와 같은 형태입니다.  
+ `input_ids`

```plain
</s>Below is an instruction that describes a task. Write a response that appropriately completes the request.

Instruction:
Give three tips for staying healthy.

Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables.
2. Exercise regularly to keep your body active and strong.
3. Get enough sleep and maintain a consistent sleep schedule.</s>
```


+ `labels`

```plain
<MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK>
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.</s>
```
`<MASK>`는 제가 임의로 적어둔 것이며, 실제로는 -100 Index 값으로 채워지게 됩니다.  
-100은 Pytorch의 모든 Loss 함수에서 `ignore_index`로 사용되는 값이므로, Loss 계산시 제외됩니다.  
따라서 `input_ids`는 모든 Sequence, `labels`는 앞쪽은 마스킹되고 연산에 사용되는 Sequence는 ChatGPT의 답변 뿐입니다.

In [8]:
# 파이토치에서 Loss 함수들의 ignore_index 값들은 모두 -100으로 처리되어 있습니다
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
IGNORE_INDEX = -100

def preprocess(
    sources,
    examples,
    tokenizer,
):
    """ 프롬프트 데이터를 받아서 전처리"""
    examples_tokenized, sources_tokenized = [tokenize(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

### 4. `dataset`과 `collator` 정의
+ 데이터셋을 로드하여 프롬프트에 포맷팅합니다.
+ 포맷팅한 Sequence를 `Dataset` 객체로 정의합니다.
+ batch 데이터를 처리하기 위해 `data_collator`를 정의합니다.
+ 데이터셋과 `data_collator`를 `trainer`에 전달하기 쉽게 `dict` 객체로 처리합니다.

In [9]:
def make_train_dataset(data_path: str):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""

    instructions = utils.jload(data_path)
    sources = []
    examples = []
    for idx in range(len(instructions)):
        question = json.loads(instructions[idx])['question']
        answer = json.loads(instructions[idx])['answer']
        source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
        example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
        sources.append(source)
        examples.append(example)

    data_dict = preprocess(sources, examples, tokenizer)
    train_dataset = Dataset.from_dict(data_dict)
    train_dataset.set_format("torch")

    return train_dataset

In [10]:
class DataCollatorForSupervisedDataset(object):
    """데이터 콜레이터"""

    def __init__(self, tokenizer):
        self.tokenizer=tokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids_ = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels_ = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids_,
            labels=labels_,
            attention_mask=input_ids_.ne(self.tokenizer.pad_token_id),
        )

In [11]:
def make_supervised_data_module(data_path: str, tokenizer):
    """학습에 사용할 데이터셋과 콜레이터를 정의합니다."""
    train_dataset = make_train_dataset(data_path)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

### 5. Lora
메모리 절감을 최대화 하기 위해 모델 내 Adapter를 추가할 수 있는 모든 Linear 연산에 추가합니다.
+ Layer이름은 모델마다 다르니 모델의 Layer를 확인하고 추가해주세요.
+ 모델의 Layer는 보통 `__repr__`로 정의되니 `model`만 입력해도 Layer가 출력됩니다.
+ `TrainingArguments`의 경우, 여러번의 학습을 통해 적절한 하이퍼파라미터를 찾아서 학습시켰습니다.  

아래 코드는 [huggingface의 블로그](https://huggingface.co/blog/gemma-pefthttps://huggingface.co/blog/gemma-peft)를 참조하였습니다.

In [12]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="gemma-2b-it-sgtuned",
    num_train_epochs=10,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    bf16=True
)

In [13]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.0,
        r=4,
        bias="none",
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

6. 학습

In [ ]:
from trl import SFTTrainer

max_seq_length = model.config.max_position_embeddings
data_module = make_supervised_data_module(data_path="/content/drive/MyDrive/Colab Notebooks/gpt_with_tuning-main/tuning/instruction.jsonl", tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    **data_module,
)
trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
10,2.539400
20,2.216800
30,1.867700
40,1.739500
50,1.639000
60,1.518400
70,1.431100
80,1.434800
90,1.295000
100,1.319600


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-662f4df3-20ea1bbd20bfd3237fcf1a8b;2850963f-72b6-440e-9eb7-b4b0f540ed4c)

Cannot access gated repo for url https://huggingface.co/google/gemma-1.1-2b-it/resolve/main/config.json.
Access to model google/gemma-1.1-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-1.1-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-1.1-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the 

In [22]:
from transformers import AutoTokenizer
import transformers
from peft import AutoPeftModelForCausalLM
import torch

my_model_id = "aiqwe/gemma-2b-it-sgtuned"
my_model = AutoPeftModelForCausalLM.from_pretrained(
    my_model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
    )
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

def generate(
    model: transformers.PreTrainedModel,
    tokenizer: transformers.PreTrainedTokenizer,
    query: str,
    rag: bool=False):

    if rag:
        documents = _scoring(query)
        documents = "\n".join(documents[:3])
        query = query +"\n아래 documents를 참조하여 답변하세요\n" + documents

    inputs = tokenizer.encode(query, add_special_tokens=False, return_tensors="pt").to(model.device)
    outputs = model.generate(input_ids=inputs, max_new_tokens=200)
    completion = tokenizer.decode(
        outputs[0],
        repetition_penalty = 1.5, # Greedy하게 답변하는것을 막기 위함
        temperature = 0.5 # Stochastic한 답변 유도
    )

    return completion

# gemma default template : https://huggingface.co/google/gemma-2b-it#chat-template
GEMMA_TRAINING_PROMPT = """<bos><start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
"""

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
GEMMA_TRAINING_PROMPT = """<bos><start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
"""
query="청약에 대해 설명해줄래"
prompt = GEMMA_TRAINING_PROMPT.format(question=query)
print(generate(model=model, tokenizer=tokenizer, query=prompt))

<bos><start_of_turn>user
청약에 대해 설명해줄래<end_of_turn>
<start_of_turn>model
**청약**은 중국과 한국의 전통적인 민속 문화와 종교적 풍부한 의상입니다. 청약은 일반적으로 청소와 정화를 의미하며, 특히 청소와 정화를 통해 신체적, 정신적, 정제적 영향을 주는 것을 의미합니다.

**청약의 역사:**

* 청약의 역사는 2000년 이전까지 거슬러 올라갑니다.
* 청약은 중국과 한국의 전통적인 민속 문화에 깊이 있게 연결되어 있습니다.
* 청약은 또한 한국의 종교적 풍부한 역사와 연결되어 있습니다.

**청약의 구성:**

* 청약에는 일반적으로 다음과 같은 구성 요소가 포함됩니다.
    * 청소
    * 정화
    * 수세
    * 샤워
    * 정


In [24]:
GEMMA_TRAINING_PROMPT = """<bos><start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
"""
query="청약에 대해 설명해줄래"
prompt = GEMMA_TRAINING_PROMPT.format(question=query)
print(generate(model=my_model, tokenizer=tokenizer, query=prompt))

<bos><start_of_turn>user
청약에 대해 설명해줄래<end_of_turn>
<start_of_turn>model
청약은 공공택지 내에서 주택을 건설하고 청약하는 방법입니다. 공공택지는 특정 조건을 충족하는 사람들에게 주택을 우선적으로 제공하는 곳입니다. 청약을 통해 주택을 받기 위해서는 특정 조건을 충족해야 하며, 일반적으로 먼저 무주택자나 실거주자이야 한 뒤에 일반인이 됩니다. 청약은 주택도시기본법에 따라 정해진 절차에 따라 진행되며, 청약통장을 가진 사람만이 청약을 할 수 있습니다. 청약통장은 특정 조건을 충족하는 사람에게 해당 공공택지 내에서 특정 기간 동안 주택을 청약할 수 있는 채권으로, 일반적으로 청약통장 1기, 2기, 3기가 있습니다. 각 채권에 따라
